In [ ]:
import geopandas as gpd
import numpy as np
from scipy.stats import weibull_min, chi2
import pandas as pd

# Read the fire data
gdf = gpd.read_file("nfdb/NFDB_point_20250519.shp")

# read in the historical fire management zones
zones = gpd.read_file("historical_fire_zones\Historical_Fire_Management_Zones.shp")

# Ensure both GeoDataFrames use the same CRS
gdf = gdf.to_crs(zones.crs)

# join the two GeoDataFrames
data = gpd.sjoin(gdf, zones, how="inner", predicate="within")

# drop the data in FMZ_DESIGN == 'Parks Zone'
data = data[data.FMZ_DESIGN != "Parks Zone"]

data.NFDBFIREID.duplicated().sum()

# show me the rows where NFDBFIREID is duplicated
duplicates = data[data.NFDBFIREID.duplicated(keep=False)]

# remove the duplicates
data = data[~data.NFDBFIREID.duplicated(keep="first")]

data = data[(data.CAUSE == "N") | (data.CAUSE == "U")]

# drop rows where the REP_DATE is null
joined = data[data.REP_DATE.notnull()]

# make a new column for whether the SIZE_HA is greater than 100\
joined["SIZE_HA_100"] = joined["SIZE_HA"].apply(lambda x: True if x > 100 else False)

# remove the rows with YEAR before 1975
joined = joined[joined.YEAR >= 1976]

# make dictionary to change values of FMZ_DESIGN
fmz_dict = {
    "Hudson Bay Zone": "Extensive",
    "Great Lakes/St. Lawrence Zone": "Intensive Measured",
    "Boreal Zone": "Intensive Measured",
    "Northern Boreal Zone": "Intensive Measured"
}

# make a new column in the joined GeoDataFrame
joined["FMZ_ZONE"] = joined["FMZ_DESIGN"].map(fmz_dict)

season_dict = {1: "Winter",
               2: "Winter",
               3: "Spring",
                4: "Spring",
                5: "Spring",
                6: "Summer",
                7: "Summer",
                8: "Summer",
                9: "Fall",
                10: "Fall",
                11: "Fall",
                12: "Winter"}

# make a new column for the season based on the column called "MONTH"
joined["SEASON"] = joined["MONTH"].map(season_dict)


# make a new column based on the column year which is whether it is after 2006 or not
def after_2005(year):
    if year > 2005:
        return "After 2005"
    else:
        return "Before 2005"

joined["AFTER_2005"] = joined["YEAR"].apply(after_2005)

joined = joined[['YEAR', 'REP_DATE', 'SIZE_HA', 'FMZ_ZONE', 'SEASON', 'AFTER_2005', 'SIZE_HA_100']]

# make a table for the total area burned by year with another column as the number of fires and the number of fires greater than 100 ha
joined_grouped = joined.groupby(['YEAR']).agg(
    {'SIZE_HA': 'sum'}).reset_index()

# now make the same columns by for the fires in the Intensive Measured zone
joined_grouped_intensive = joined[joined.FMZ_ZONE == "Intensive Measured"].groupby(['YEAR']).agg(
    {'REP_DATE':'count', 'SIZE_HA_100': 'sum'}).reset_index()

# rename the columns
joined_grouped_intensive = joined_grouped_intensive.rename(
    columns={'REP_DATE': 'Number of Fires in the IM Zones', 'SIZE_HA_100': 'Number of Large Fires in the IM Zones'})

# do the same for the Extensive zone
joined_grouped_extensive = joined[joined.FMZ_ZONE == "Extensive"].groupby(['YEAR']).agg(
    {'REP_DATE':'count', 'SIZE_HA_100': 'sum'}).reset_index()

# rename the columns
joined_grouped_extensive = joined_grouped_extensive.rename(
    columns={'REP_DATE': 'Number of Fires in the Ext Zones', 'SIZE_HA_100': 'Number of Large Fires in the Ext Zones'})

# now merge the two dataframes
joined_grouped = pd.merge(joined_grouped, joined_grouped_intensive, on='YEAR', how='left')

# now merge the two dataframes

joined_grouped = pd.merge(joined_grouped, joined_grouped_extensive, on='YEAR', how='left')

joined_grouped